In [119]:
import xgboost as xgb
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from tensorflow.keras.models import Sequential
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Input, Dropout, Conv1D, Flatten, GlobalMaxPool1D, Reshape, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.model_selection import GridSearchCV
from kerastuner.tuners import RandomSearch

In [120]:
#Pasamos Los Features del otro notebook

oportunidades = pd.read_csv("Train_TP2_Datos_2020-2C.csv")
oportunidades['Account_Created_Date'] = pd.to_datetime(oportunidades['Account_Created_Date'])
oportunidades['Opportunity_Created_Date'] = pd.to_datetime(oportunidades['Opportunity_Created_Date'])
oportunidades['Quote_Expiry_Date'] = pd.to_datetime(oportunidades['Quote_Expiry_Date'])
oportunidades['Last_Modified_Date'] = pd.to_datetime(oportunidades['Last_Modified_Date'])
oportunidades['Planned_Delivery_Start_Date'] = pd.to_datetime(oportunidades['Planned_Delivery_Start_Date'])
oportunidades['Planned_Delivery_End_Date'] = pd.to_datetime(oportunidades['Planned_Delivery_End_Date'])
oportunidades_japon = (oportunidades.loc[oportunidades['Region'] == 'Japan'])
oportunidades = (oportunidades.loc[oportunidades['Region'] != 'Japan'])
#oportunidades_japon = oportunidades_japon.iloc[:,:3]
oportunidades_japon['Territory'] = oportunidades_japon['Territory'].replace({'None':'Japan'})
oportunidades = pd.concat([oportunidades, oportunidades_japon], axis=0)
oportunidades['Region'] = oportunidades['Region'].replace({'Japan':'APAC', 'Middle East':'EMEA'})
oportunidades[oportunidades.select_dtypes(['object']).columns] = oportunidades.select_dtypes(['object']).apply(lambda x: x.astype('category'))
oportunidades = oportunidades.loc[oportunidades['Territory'] != 'None']
oportunidades = oportunidades.dropna()

filtro_terminos_entrega = oportunidades['Pricing, Delivery_Terms_Quote_Appr']\
     == oportunidades['Pricing, Delivery_Terms_Approved']

filtro_codigo_burocratico = oportunidades['Bureaucratic_Code_0_Approval']\
     == oportunidades['Bureaucratic_Code_0_Approved']

oportunidades = oportunidades.drop(['Pricing, Delivery_Terms_Quote_Appr'\
    ,'Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval'\
        , 'Bureaucratic_Code_0_Approved', 'Submitted_for_Approval'], axis = 'columns')

oportunidades_posibles = (filtro_terminos_entrega & filtro_codigo_burocratico)

oportunidades.insert(3,'Es_Oportunidad_Posible', oportunidades_posibles)

oportunidades['Es_Oportunidad_Posible'] = oportunidades['Es_Oportunidad_Posible'].replace(
    {True:1, False:0})
oportunidades.head()

encoding_owners = oportunidades.groupby('Opportunity_Owner').agg({'Total_Amount':'mean'})
encoding_owners.columns = ['Encoding_Vendedor']
encoding_owners = encoding_owners.reset_index()
oportunidades = oportunidades.merge(encoding_owners, how='inner', on='Opportunity_Owner')

encoding_territory = oportunidades.groupby('Territory').agg({'Total_Amount' : 'mean'})
encoding_territory.columns = ['Encoding_Territorio']
encoding_territory = encoding_territory.reset_index()
oportunidades = oportunidades.merge(encoding_territory, how='inner', on='Territory')

oportunidades = pd.concat([oportunidades, pd.get_dummies(oportunidades['Region'])],axis='columns')

encoding_moneda = oportunidades.groupby('Total_Amount_Currency').agg({'ASP' : 'mean'})
encoding_moneda.columns = ['Encoding_Moneda']
encoding_moneda = encoding_moneda.reset_index()
oportunidades = oportunidades.merge(encoding_moneda, how='inner', on='Total_Amount_Currency')

oportunidades = pd.concat([oportunidades, pd.get_dummies(oportunidades['Bureaucratic_Code'])], axis='columns')

encoding_bill_c = oportunidades.groupby('Billing_Country').agg({'Total_Amount' : 'mean'})
encoding_bill_c.columns = ['Encoding_Billing_Country']
encoding_bill_c = encoding_bill_c.reset_index()
oportunidades = oportunidades.merge(encoding_bill_c, how='inner', on='Billing_Country')

encoding_product_family = oportunidades.groupby('Product_Family').agg({'Total_Amount' : 'mean'})
encoding_product_family.columns = ['Encoding_Prod_Family']
encoding_product_family = encoding_product_family.reset_index()
oportunidades = oportunidades.merge(encoding_product_family, how='inner', on='Product_Family')

oportunidades = pd.concat([oportunidades, pd.get_dummies(oportunidades['Account_Type'])],axis='columns')

encoding_account_name = oportunidades.groupby('Account_Name').agg({'Total_Amount' : 'mean'})
encoding_account_name.columns = ['Encoding_Acc_Name']
encoding_account_name = encoding_account_name.reset_index()
oportunidades = oportunidades.merge(encoding_account_name, how='inner', on='Account_Name')

oportunidades = pd.concat([oportunidades,pd.get_dummies(oportunidades['Delivery_Terms'])],axis='columns')

oportunidades['Opportunity_Created_Year'] = oportunidades['Opportunity_Created_Date'].dt.year
oportunidades['Opportunity_Created_Month'] = oportunidades['Opportunity_Created_Date'].dt.month

encoding_op_type = oportunidades.groupby('Opportunity_Type').agg({'Total_Taxable_Amount' : 'mean'})
encoding_op_type.columns = ['Encoding_Opportunity_Type']
encoding_op_type = encoding_op_type.reset_index()
oportunidades = oportunidades.merge(encoding_op_type, how='inner', on='Opportunity_Type')

oportunidades = pd.concat([oportunidades, pd.get_dummies(oportunidades['Source '])],axis = 'columns')

y_train = oportunidades['Stage']
oportunidades = oportunidades.loc[(oportunidades['Stage'] == 'Closed Won') | (oportunidades['Stage'] == 'Closed Lost')]
oportunidades['Stage'] = oportunidades['Stage'].replace({'Closed Won': 1, 'Closed Lost': 0})
std_deviation_amounts = oportunidades.groupby('Opportunity_ID').agg({'TRF':'std', 'Opportunity_Created_Month':'std'})
std_deviation_amounts.columns = ['TRF_Std', 'Opp_Created_Month_Std']
std_deviation_amounts = std_deviation_amounts.reset_index()

#Solo usamos Bureau_Code_ 0,1,2,4,5 porque son los que tiene el set de test.
x_train = oportunidades.groupby('Opportunity_ID').agg({'Opportunity_Created_Year' : 'mean',\
                                                       'Opportunity_Created_Month' : 'mean',\
                                                      'Encoding_Moneda':'mean', 'Total_Amount': 'sum',\
                                                      'Total_Taxable_Amount':'sum',\
                                                      'Encoding_Territorio':'mean','Encoding_Vendedor':'mean',\
                                                      'Es_Oportunidad_Posible':'mean','Stage':'max', 'TRF':'mean',
                                                      'Bureaucratic_Code_0':'mean','Bureaucratic_Code_1':'mean',\
                                                      'Bureaucratic_Code_2':'mean','Bureaucratic_Code_4':'mean',
                                                      'Bureaucratic_Code_5':'mean', 'Encoding_Billing_Country':'mean',\
                                                      'Delivery_Terms_0':'mean','Delivery_Terms_1':'mean','Delivery_Terms_2':'mean',\
                                                      'Delivery_Terms_3':'mean','Delivery_Terms_4':'mean','Delivery_Terms_5':'mean',\
                                                      'Delivery_Terms_6':'mean','Delivery_Terms_7':'mean','Delivery_Terms_8':'mean',
                                                      'Encoding_Prod_Family':'mean','Account_Type_0':'mean','Account_Type_1':'mean',\
                                                      'Account_Type_2':'mean','Account_Type_4':'mean', 'Account_Type_5':'mean',\
                                                      'APAC' : 'mean','Americas':'mean','EMEA':'mean', 'Encoding_Acc_Name':'mean',\
                                                      'Encoding_Opportunity_Type':'mean', 'Source_3':'mean', 'Source_7':'mean',\
                                                      'Source_9':'mean','Source_11':'mean','Source_13':'mean'})
x_train = x_train.reset_index()
x_train = x_train.merge(std_deviation_amounts, on='Opportunity_ID', how='inner')
x_train = x_train.set_index('Opportunity_ID')
y_train = x_train['Stage']
x_train = x_train.drop('Stage', axis='columns')
x_train['TRF_Std'] = x_train['TRF_Std'].replace({np.nan:0})
x_train['Opp_Created_Month_Std'] = x_train['Opp_Created_Month_Std'].replace({np.nan:0})

In [121]:
#Creamos los modelos de XGBoost y CatBoost

params = {
    'objective':'binary:logistic',
    'max_depth':12,
    'n_estimators':450,
    'colsample_bytree':0.3,
    'subsample':0.7,
    'learning_rate':0.014,
    'alpha':0.01,
    'gamma':0.0
}

params2 = {
    'objective':'binary:logistic',
    'max_depth':9,
    'n_estimators':350,
    'colsample_bytree':0.5,
    'subsample':0.5,
    'learning_rate':0.02,
    'alpha':0.0001,
    'gamma':0.1
}

xg_reg = xgb.XGBClassifier(objective =params['objective'], 
                colsample_bytree = params['colsample_bytree'], learning_rate = params['learning_rate'],
                max_depth = params['max_depth'], alpha = params['alpha'], n_estimators = params['n_estimators'], 
                subsample = params['subsample'], early_stopping_rounds=5)

xg_reg2 = xgb.XGBClassifier(objective =params2['objective'], 
                colsample_bytree = params2['colsample_bytree'], learning_rate = params2['learning_rate'],
                max_depth = params2['max_depth'], alpha = params2['alpha'], n_estimators = params2['n_estimators'], 
                subsample = params2['subsample'], early_stopping_rounds=5)

model_xgb = xgb.XGBClassifier()
model_catb = CatBoostClassifier(verbose = False)

In [122]:
#Dividimos el set en train y test, y entrenamos ambos modelos

division_x_train, division_x_test, division_y_train, division_y_test = train_test_split(x_train, y_train, test_size = 0.3,\
                                                                                       random_state = 123)
model_xgb.fit(division_x_train, division_y_train)
model_catb.fit(division_x_train, division_y_train)
xg_reg.fit(division_x_train, division_y_train)
xg_reg2.fit(division_x_train, division_y_train)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:21:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:21:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:21:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore

XGBClassifier(alpha=0.0001, base_score=0.5, booster='gbtree',
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
              early_stopping_rounds=5, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.02, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=350, n_jobs=12, num_parallel_tree=1, random_state=0,
              reg_alpha=9.99999975e-05, reg_lambda=1, scale_pos_weight=1,
              subsample=0.5, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [123]:
#Scoreamos con XGBoost
score_xgb = model_xgb.score(division_x_test, division_y_test)
log_loss_xgb = log_loss(division_y_test,model_xgb.predict_proba(division_x_test))
print("Score = "+str(score_xgb)+" ; Loss Original = " + str(log_loss_xgb))

Score = 0.812793979303857 ; Loss Original = 0.41059451227319393


In [124]:
#Scoreamos con CatBoost
score_cb = model_catb.score(division_x_test, division_y_test)
loss_cb = log_loss(division_y_test, model_catb.predict_proba(division_x_test))
print("Score = "+str(score_cb)+" ; Loss Original = " + str(loss_cb))

Score = 0.819379115710254 ; Loss Original = 0.3942153799983715


In [125]:
#Scoreamos con XGBoost tuneado
score = xg_reg.score(division_x_test, division_y_test)
log_loss_original = log_loss(division_y_test, xg_reg.predict_proba(division_x_test))
array = xg_reg.predict_proba(division_x_test)
#array = pd.DataFrame(array)
for i in range(len(array)):
    if array[i][1] <= 0.025:
        array[i][1] = 0.025
    elif array[i][1] >= 0.975:
        array[i][1] = 0.975
df = pd.DataFrame(array)
array = df.loc[:,1]
log_loss_mod = log_loss(division_y_test, array)
print("Score = "+str(score)+" ; Loss Original = " + str(log_loss_original) + " ; Loss modificado = " + str(log_loss_mod))

Score = 0.8189087488240828 ; Loss Original = 0.3873882698024836 ; Loss modificado = 0.3877078456416908


In [126]:
#Con segundo XGBoost tuneado
score2 = xg_reg2.score(division_x_test, division_y_test)
log_loss_original2 = log_loss(division_y_test, xg_reg2.predict_proba(division_x_test))
array2 = xg_reg2.predict_proba(division_x_test)
#array = pd.DataFrame(array)
for i in range(len(array2)):
    if array2[i][1] <= 0.025:
        array2[i][1] = 0.025
    elif array2[i][1] >= 0.975:
        array2[i][1] = 0.975
df2 = pd.DataFrame(array2)
array2 = df2.loc[:,1]
log_loss_mod2 = log_loss(division_y_test, array2)
print("Score = "+str(score2)+" ; Loss Original = " + str(log_loss_original2) + " ; Loss modificado = " + str(log_loss_mod2))

Score = 0.8165569143932268 ; Loss Original = 0.387051545769968 ; Loss modificado = 0.387338173365429
